##Classification

In [1]:
!pip install --upgrade tensorflow==1.15

  Using cached https://files.pythonhosted.org/packages/92/2b/e3af15221da9ff323521565fa3324b0d7c7c5b1d7a8ca66984c8d59cb0ce/tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [3]:
def load_dataset():
  df = pd.read_csv('classification.csv')

  class_conv = []
  shape = []
  odor_conv = []
  habitat_conv = []
  df = df[['class','cap-shape','cap-color','odor','stalk-color-above-ring','stalk-color-below-ring','veil-color','ring-number','habitat']]

  for item in df['class']:
    if item == 'p':
      class_conv.append(1)
    else:
      class_conv.append(0)
  df['class'] = class_conv

  for item in df['cap-shape']:
    if item == 'b':
      shape.append(1)
    elif item == 'c':
      shape.append(2)
    elif item == 'x':
      shape.append(3)
    elif item == 'f':
      shape.append(4)
    elif item == 'k':
      shape.append(5)
    elif item == 's':
      shape.append(6)
  df['cap-shape'] = shape

  for item in df['odor']:
    if item == 'a':
      odor_conv.append(1)
    elif item == 'l':
      odor_conv.append(2)
    elif item == 'c':
      odor_conv.append(3)
    elif item == 'y':
      odor_conv.append(4)
    elif item == 'f':
      odor_conv.append(5)
    elif item == 'm':
      odor_conv.append(6)
    elif item == 'n':
      odor_conv.append(7)
    elif item == 'p':
      odor_conv.append(8)
    elif item == 's':
      odor_conv.append(9)
  df['odor'] = odor_conv

  for item in df['habitat']:
    if item == 'g':
      habitat_conv.append(1)
    elif item == 'l':
      habitat_conv.append(2)
    elif item == 'm':
      habitat_conv.append(3)
    elif item == 'p':
      habitat_conv.append(4)
    elif item == 'u':
      habitat_conv.append(5)
    elif item == 'w':
      habitat_conv.append(6)
    elif item == 'd':
      habitat_conv.append(7)
  df['habitat'] = habitat_conv

  return df

In [4]:
dataset = load_dataset()
print(dataset.head(5))

   class  cap-shape  cap-color  ...  veil-color  ring-number  habitat
0      1          3          4  ...           2            1        5
1      0          3          9  ...           2            1        1
2      0          1          8  ...           2            1        3
3      1          3          8  ...           2            1        5
4      0          3          3  ...           2            1        1

[5 rows x 9 columns]


In [5]:
feature = dataset[['cap-shape','cap-color','odor','stalk-color-above-ring','stalk-color-below-ring','veil-color','ring-number','habitat']].values
label = dataset['class'].values.reshape(-1,1)

In [6]:
label

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [7]:
# scaler = StandardScaler()
# dataset = scaler.fit_transform(dataset)

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(feature)
principalData = pd.DataFrame(data = principalComponents, columns = ['PC1','PC2','PC3','PC4'])

In [9]:
dataset = pd.concat([principalData, dataset[['class']]], axis = 1)

In [10]:
dataset.head(5)

,PC1,PC2,PC3,PC4,class
0,-1.121878,-1.690867,-0.819994,-1.883650,1
1,7.058268,1.033189,-1.439619,1.880202,0
2,5.013362,1.194147,-2.310757,1.358583,0
3,1.537280,0.302015,-1.517290,-3.980871,1
4,0.999276,-3.636269,1.605544,0.070107,0


In [11]:
X = dataset[['PC1','PC2','PC3','PC4']].values
Y = dataset['class'].values.reshape(-1,1)

In [12]:
#SPLIT DATA
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.3)

In [13]:
#PREPROCESSING FEATURES
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [14]:
encoder = OneHotEncoder().fit(y_train)

y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

In [15]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [16]:
epochs = 2500
learning_rate = 0.01
n_features = x_train[0].shape[0]
n_classes = y_train[0].shape[0]

In [17]:
#neural network
input_tensor = tf.placeholder(tf.float32, name = 'input')
label_tensor = tf.placeholder(tf.float32, name = 'label')

In [18]:
neurons = [2, 64, 2]
n_layers = len(neurons)

In [19]:
parameters = {
    'W1' : tf.Variable(
        tf.random.normal([
            n_features, neurons[0]
        ]),
        tf.float32
    ),
    'B1' : tf.Variable(
        tf.random.normal([1,  neurons[0]]),
        tf.float32
    ),
    'W2' : tf.Variable(
        tf.random.normal([
            neurons[0], neurons[1]
        ]),
        tf.float32
    ),
    'B2' : tf.Variable(
        tf.random.normal([1,  neurons[1]]),
        tf.float32
    ),
    'W3' : tf.Variable(
        tf.random.normal([
            neurons[1], neurons[2]
        ]),
        tf.float32
    ),
    'B3' : tf.Variable(
        tf.random.normal([1,  neurons[2]]),
        tf.float32
    )
}

In [20]:
#FORWARD PASS
def forward_pass(input_tensor,params,n_layer=3):
  a = input_tensor
  for i in range(n_layer):
    W = params[f'W{i+1}']
    B = params[f'B{i+1}']

    z = tf.matmul(a,W) + B 
    if i == n_layer-1:
      a = tf.nn.sigmoid(z)
    else:
      a = tf.nn.sigmoid(z)

  return a 

In [21]:
def evaluate_accuracy(y, yhat):
  yield (0.5 * (yhat - y)**2).sum(axis=1).mean()
  yield accuracy_score(np.argmax(y, axis = 1), np.argmax(yhat, axis = 1))

In [22]:
# with tf.Session() as sess:
#   logits_tensor = forward_pass(input_tensor, parameters)
#   loss_tensor = tf.losses.mean_squared_error(label_tensor, logits_tensor)
#   true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
#   acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

#   optimiser = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_tensor)
#   sess.run(tf.global_variables_initializer())
#   for i in range(epochs):
#     sess.run(
#         optimiser,
#         feed_dict = {
#             input_tensor: x_train,
#             label_tensor: y_train
#         }
#     )

#     if i%250 == 0:
#       loss,acc = evaluate_accuracy(y_test,sess.run(
#           logits_tensor,
#           feed_dict = {
#               input_tensor: x_test,
#               label_tensor: y_test
#           }
#       ))
#       print(f'Epochs: ',i, ' ||Loss: ',loss, ' ||Accuracy: ', {acc*100},'%')

In [23]:
with tf.Session() as sess: #jawban
  logits_tensor = forward_pass(input_tensor, parameters)
  loss_tensor = tf.losses.mean_squared_error(label_tensor, logits_tensor)
  true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
  acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

  optimiser = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_tensor)
  sess.run(tf.global_variables_initializer())

  test_dict = {
      input_tensor: x_test,
      label_tensor: y_test
  }
  best_loss = float('inf')

  for i in range(epochs):
    sess.run(
        optimiser,
        feed_dict = {
            input_tensor: x_train,
            label_tensor: y_train
        }
    )

    if i%25 == 0:
      loss,acc = evaluate_accuracy(y_test,sess.run(
          logits_tensor,
          feed_dict = {
              input_tensor: x_test,
              label_tensor: y_test
          }
      ))
      print(f'Epochs: ',i, ' ||Loss: ',loss, ' ||Accuracy: ', {acc*100},'%')

    if i %125 == 0:
      # loss = sess.run(optimizer, feed_dict=train_dict)
      val_loss = sess.run(loss_tensor, feed_dict = test_dict)
      print(f'Epoch {i+1} - val_loss {val_loss}')

      if val_loss < best_loss:
        best_loss = val_loss
        saver.save(sess, 'best_model.ckpt')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epochs:  0  ||Loss:  0.47551244  ||Accuracy:  {50.7383100902379} %
Epoch 1 - val_loss 0.47551244497299194


NameError: ignored